<a href="https://colab.research.google.com/github/thaitruong018/e_commerce_analysis/blob/main/5_Customer_lifetime_value.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## What is Customer lifetime value?
Customer lifetime value (CLTV) is represents the total amount of money a customer is expected to spend in a business during his/her lifetime.  
There are few approaches available to compute CLTV.  
In my notebook, I use a simple approach:  
CLTV = ((Average Order Value or Sales x Frequency of purchase)/Churn Rate) x Profit Margin  
 - Average order value = Total Revenue / Total Number of Orders
 - Frequency of purchase = Total Number of Orders / Total Number of Customers  
 - Churn rate = 1 - Repeat rate  
 - Profit margin represents what percentage of sales turn into profits
 - Repeat rate can be defined as the ratio of the number of customers with more than one order and the number of unique customers

## Set-up

In [47]:
# connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [48]:
# import libraries
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

In [49]:
# path to data
path = '/content/gdrive/MyDrive/brazilian_ecommerce'

## Data

In [50]:
# read data
data = pd.read_csv(os.path.join(path,'df_order_items_products_customers.csv'))
data.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_id,quantity,seller_id,shipping_limit_date,price,freight_value,product_category_name,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,2e7a8482f6fb09756ca50c10d7bfc047,08c5351a6aca1c1589a38f244edeee9d,shipped,2016-09-04,2016-10-07,2016-10-18,2016-11-09,2016-10-20,c1488892604e4ba5cff5b4eb4d595400,1,1554a68530182680ad5c8b042c3ab563,2016-10-26 18:25:19,39.99,31.67,moveis_decoracao,b7d76e111c89f7ebf14761390f0f7d17,69309,boa vista,RR
1,2e7a8482f6fb09756ca50c10d7bfc047,08c5351a6aca1c1589a38f244edeee9d,shipped,2016-09-04,2016-10-07,2016-10-18,2016-11-09,2016-10-20,f293394c72c9b5fafd7023301fc21fc2,1,1554a68530182680ad5c8b042c3ab563,2016-10-26 18:25:19,32.90,31.67,moveis_decoracao,b7d76e111c89f7ebf14761390f0f7d17,69309,boa vista,RR
2,35d3a51724a47ef1d0b89911e39cc4ff,27ab53f26192510ff85872aeb3759dcc,delivered,2016-10-04,2016-10-05,2016-10-14,2016-10-26,2016-12-20,c1488892604e4ba5cff5b4eb4d595400,1,1554a68530182680ad5c8b042c3ab563,2016-10-14 02:44:30,39.99,22.34,moveis_decoracao,f922896769e9517ea3c630f3c8de86d0,96832,santa cruz do sul,RS
3,c4f710df20f7d1500da1aef81a993f65,4b671f05b6eb9dc1d2c1bae9c8c78536,delivered,2016-10-10,2016-10-10,2016-10-18,2016-10-26,2016-12-14,c1488892604e4ba5cff5b4eb4d595400,1,1554a68530182680ad5c8b042c3ab563,2016-10-26 16:35:46,39.99,11.80,moveis_decoracao,0ecf7f65b5ff3b9e61b637e59f495e0a,30411,belo horizonte,MG
4,81e5043198a44ddeb226002ff55d8ad4,ddd15ef77c83eea8c534d2896173a927,delivered,2017-01-09,2017-01-09,2017-01-09,2017-02-24,2017-02-24,c1488892604e4ba5cff5b4eb4d595400,2,1554a68530182680ad5c8b042c3ab563,2017-01-13 00:37:18,38.50,15.56,moveis_decoracao,853ba75a0b423722ccf270eea3b4cfe4,21843,rio de janeiro,RJ


In [51]:
# create new dataframe to calculate cltv
df_clv = data[['order_id','customer_unique_id','product_id','order_purchase_timestamp',
                    'quantity','price'
                    ]].copy()
df_clv.rename(columns={'order_purchase_timestamp':'purchased_date'},inplace=True)
df_clv.head()

,order_id,customer_unique_id,product_id,purchased_date,quantity,price
0,2e7a8482f6fb09756ca50c10d7bfc047,b7d76e111c89f7ebf14761390f0f7d17,c1488892604e4ba5cff5b4eb4d595400,2016-09-04,1,39.99
1,2e7a8482f6fb09756ca50c10d7bfc047,b7d76e111c89f7ebf14761390f0f7d17,f293394c72c9b5fafd7023301fc21fc2,2016-09-04,1,32.90
2,35d3a51724a47ef1d0b89911e39cc4ff,f922896769e9517ea3c630f3c8de86d0,c1488892604e4ba5cff5b4eb4d595400,2016-10-04,1,39.99
3,c4f710df20f7d1500da1aef81a993f65,0ecf7f65b5ff3b9e61b637e59f495e0a,c1488892604e4ba5cff5b4eb4d595400,2016-10-10,1,39.99
4,81e5043198a44ddeb226002ff55d8ad4,853ba75a0b423722ccf270eea3b4cfe4,c1488892604e4ba5cff5b4eb4d595400,2017-01-09,2,38.50


In [52]:
# change datatype
df_clv['purchased_date'] = pd.to_datetime(df_clv['purchased_date'])

In [53]:
# calculate total payment
df_clv['TotalPurchase'] = df_clv['quantity']*df_clv['price']
# calculation necessary attributes
df_clv_group = df_clv.groupby('customer_unique_id').agg({'purchased_date':lambda date: (date.max()-date.min()).days,  # calculate lifetime
                                                   'order_id': 'count', # number of orders
                                                   'quantity': 'sum', # total quantity
                                                   'TotalPurchase': 'sum'             # total value                                      
                                                   })
# rename columns
df_clv_group.columns=['num_days','num_transactions','num_units','total_purchase']
df_clv_group.head()

,num_days,num_transactions,num_units,total_purchase
customer_unique_id,,,,
0000366f3b9a7992bf8c76cfdf3221e2,0,1,1,129.90
0000b849f77a49e4a4ce2b2a4ca5be3f,0,1,1,18.90
0000f46a3911fa3c0805444483337064,0,1,1,69.00
0000f6ccb0745a6a4b88665a16c9f078,0,1,1,25.99
0004aac84e0df4da2b147fca70cf8255,0,1,1,180.00


## Calculate CLTV

In [54]:
# calculate Average order value
df_clv_group['avg_order_value'] = df_clv_group['total_purchase']/df_clv_group['num_transactions']
# calculate frequency of purchase
purchase_frequency=sum(df_clv_group['num_transactions'])/df_clv_group.shape[0]
# calculate repeat rate
repeat_rate = df_clv_group[df_clv_group['num_transactions']>1].shape[0]/df_clv_group.shape[0]
# calculate churn rate
churn_rate = 1 -repeat_rate
# profit margin: assume company gain 5% profit 
df_clv_group['profit_margin']=df_clv_group['total_purchase']*0.05
# calculate customer value
df_clv_group['CLV'] = (df_clv_group['avg_order_value']*purchase_frequency)/churn_rate
# calculate customer lifetime value
df_clv_group['cust_lifetime_value'] = df_clv_group['CLV']*df_clv_group['profit_margin']

df_clv_group.head()

,num_days,num_transactions,num_units,total_purchase,avg_order_value,profit_margin,CLV,cust_lifetime_value
customer_unique_id,,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,0,1,1,129.90,129.90,6.4950,148.442029,964.130978
0000b849f77a49e4a4ce2b2a4ca5be3f,0,1,1,18.90,18.90,0.9450,21.597801,20.409922
0000f46a3911fa3c0805444483337064,0,1,1,69.00,69.00,3.4500,78.849115,272.029446
0000f6ccb0745a6a4b88665a16c9f078,0,1,1,25.99,25.99,1.2995,29.699833,38.594933
0004aac84e0df4da2b147fca70cf8255,0,1,1,180.00,180.00,9.0000,205.693343,1851.240084


## Predict CLTV

In [55]:
# extract purchase year month
df_clv['PurchaseYearMonth'] = df_clv['purchased_date'].dt.to_period('M')
# create a pivot table 
revenue = pd.pivot_table(data= df_clv, 
                      columns='PurchaseYearMonth',
                      index='customer_unique_id',
                      values='TotalPurchase',
                      aggfunc = 'sum', fill_value =0)
revenue.head()

PurchaseYearMonth,2016-09,2016-10,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09
customer_unique_id,,,,,,,,,,,,,,,,,,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,129.9,0.0,0.0,0.0,0
0000b849f77a49e4a4ce2b2a4ca5be3f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,18.9,0.0,0.0,0.0,0
0000f46a3911fa3c0805444483337064,0.0,0.0,0.0,0.0,0.0,69.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
0000f6ccb0745a6a4b88665a16c9f078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
0004aac84e0df4da2b147fca70cf8255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [56]:
# cal CLV column
revenue['CLV']=revenue.sum(axis=1)
revenue = revenue.reset_index()
revenue.head()

PurchaseYearMonth,customer_unique_id,2016-09,2016-10,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,CLV
0,0000366f3b9a7992bf8c76cfdf3221e2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,129.9,0.0,0.0,0.0,0,129.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,18.9,0.0,0.0,0.0,0,18.90
2,0000f46a3911fa3c0805444483337064,0.0,0.0,0.0,0.0,0.0,69.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,69.00
3,0000f6ccb0745a6a4b88665a16c9f078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,25.99
4,0004aac84e0df4da2b147fca70cf8255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,180.00


In [57]:
# define input, output
X= revenue.iloc[:,4:-1] # select the latest 21 months as input
y= revenue[['CLV']] # output

In [58]:
#split training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 ,random_state=42)

In [59]:
# import model
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [60]:
# calculate MAE using scikit-learn
print("MAE:",mean_absolute_error(y_test,y_pred))#calculate mean squared error
print("MSE",mean_squared_error(y_test, y_pred))
# compute the RMSE of our predictions
print("RMSE:",np.sqrt(mean_squared_error(y_test, y_pred)))

MAE: 1.050603225676365
MSE 123.51242694121925
RMSE: 11.113614485900582
